# 倒立振子のシミュレーションを Julia で行いたい

## はじめに

### 倒立振子とは

台車の上に棒が立っていて，その棒が倒れないようにしつつ台車を動かすという制御問題のことを倒立振子（とうりつしんし，inverted pendulum）といいます．

制御や強化学習の界隈の Hello World としてよく親しまれている問題です．倒立振子のシミュレーションを自分で組むと勉強になるとされています．

### Julia とは

主に科学計算の世界で使われている比較的新しい言語です．Python や Matlab を使ってもいいのですが，今回は Julia を使います．

## 倒立振子の運動方程式

シミュレーションをするには，運動方程式がわかっている必要があります．

運動方程式の導出は物理学の問題であって，制御の問題ではないのでここでは詳しくは述べません．[Excelで学ぶ振動基礎/運動方程式の立て方](https://edu.katzlab.jp/lec/vib7h/files/vib7h_B.pdf)に解析力学を使って導出する方法が解説されているので，そちらを参照してください．

ここでは以下のようなパラメータを考えます．

* 定数
  * 台車の質量 $M$ [kg]
  * 振子の質量 $m$ [kg]
  * 重力加速度 $g$ [m/s^2]
  * 台車から振子の重心までの距離 $l$ [m]

* 変数
  * 台車の位置 $x$ [m]
  * 振子の傾き $θ$ [rad]
  * 外力 $f$ [N]

摩擦や空気抵抗は考えないものとしています．

このとき，結論だけ書いてしまうと，運動方程式は次の通りです．
$$
(M + m) \ddot{x} + m l \ddot{θ} \cos θ - m l \dot{θ}^2 \sin θ = f \\\\
(ml \cos θ) \ddot{x} + ml^2 \ddot{θ} + mgl \sin θ = 0
$$

## 線形化する

非線形なままだと扱いづらくて困るので，似ている線形システムに置き換えてしまいます．ここでは $θ$ が小さいと仮定して線形化します．これは，振子の傾きがそんなに大きくないということを意味します．このとき，$\sin θ ≈ θ, \cos θ ≈ 1$ と近似できるので，運動方程式は次のようになります．

<!-- ## 状態方程式を求める

さらにこれを状態空間モデルとして解釈するために，状態ベクトル $X$ を考えます．状態は $x, \dot{x}, θ, \dot{θ}$ の 4 つなので
$$
X = \begin{bmatrix} x \\ \dot{x} \\ θ \\ \dot{θ} \end{bmatrix}
$$
とします． -->

In [5]:
print("Hello World!")

Hello World!